<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/LeNet_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Upload kaggle.json
from google.colab import files
files.upload()  # Choose your kaggle.json file


{}

In [7]:
# Make a hidden directory for Kaggle and move kaggle.json there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Give permission
!chmod 600 ~/.kaggle/kaggle.json

# Test if Kaggle API is working
# !kaggle datasets list


In [8]:
!kaggle datasets download engadhamibrahim/mri-brain-tumor


Dataset URL: https://www.kaggle.com/datasets/engadhamibrahim/mri-brain-tumor
License(s): CC0-1.0


In [9]:
!unzip '/content/mri-brain-tumor.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1655).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1656).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1657).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1658).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1659).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1660).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1661).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1662).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1663).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1664).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1665).jpg  
  inflating: Brain-tumer/validation/meningioma_tumor/menin_tumor(1666).jpg  
  inflating: Brain-tumer/

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and scaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2  # 20% validation
)

# Load from directory
train_generator = train_datagen.flow_from_directory(
    '/content/Brain-tumer',  # path to your dataset
    target_size=(32, 32),
    color_mode='grayscale',
    batch_size=64,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/Brain-tumer',
    target_size=(32, 32),
    color_mode='grayscale',
    batch_size=64,
    class_mode='sparse',
    subset='validation'
)

# Now use the same LeNet-5 model from before
# model.fit(train_generator, validation_data=validation_generator, epochs=15)


Found 27572 images belonging to 3 classes.
Found 6891 images belonging to 3 classes.


In [14]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(32,32,1)),  # Correct way to define input
    layers.Conv2D(6, (5,5), activation='tanh', padding='same'),
    layers.AveragePooling2D(pool_size=(2,2)),

    layers.Conv2D(16, (5,5), activation='tanh', padding='valid'),
    layers.AveragePooling2D(pool_size=(2,2)),

    layers.Flatten(),
    layers.Dense(120, activation='tanh'),
    layers.Dense(84, activation='tanh'),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Adapt to your dataset
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 16, 16, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 6, 6, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │        69,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,231 (321.21 KB)

 Trainable params: 82,231 (321.21 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(train_generator, validation_data=validation_generator, epochs=15)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 84s 191ms/step - accuracy: 0.6003 - loss: 0.9598 - val_accuracy: 0.5999 - val_loss: 0.9523
Epoch 2/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 144s 197ms/step - accuracy: 0.5994 - loss: 0.9529 - val_accuracy: 0.5999 - val_loss: 0.9519
Epoch 3/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 82s 189ms/step - accuracy: 0.6006 - loss: 0.9513 - val_accuracy: 0.5999 - val_loss: 0.9525
Epoch 4/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 87s 201ms/step - accuracy: 0.6037 - loss: 0.9472 - val_accuracy: 0.5999 - val_loss: 0.9517
Epoch 5/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 82s 190ms/step - accuracy: 0.5992 - loss: 0.9517 - val_accuracy: 0.5999 - val_loss: 0.9505
Epoch 6/15
431/431 ━━━━━━━━━━━━━━━━━━━━ 80s 187ms/step - accuracy: 0.6026 - loss: 0.9482 - val_accuracy: 0.5999 - val_loss: 0.9507
Epoch 7/15
241/431 ━━━━━━━━━━━━━━━━━━━━ 29s 155ms/step - accuracy: 0.6013 - loss: 0.9495

KeyboardInterrupt: 